# Flappy Bird

## Installations

In [ ]:
# Using py game -> visuals
!pip install pygame

In [ ]:
!pip install neat-python

## Code

<B><U> 3 classes </U> </B> 
   * Pipe
   * Bird
   * Ground

In [13]:
# imports

import pygame
import neat
import time 
import os
import random #for randomly placing height of tubes
pygame.font.init()

In [14]:
# All number or constant names of variables in capital

WIN_WIDTH = 500
WIN_HEIGHT = 800

In [15]:
#loading images
#scale2x -> makes image 2 times bigger
#load -> loads an image

GEN = 0

BIRD_IMGS = [pygame.transform.scale2x(pygame.image.load(os.path.join("imgs","bird1.png"))), pygame.transform.scale2x(pygame.image.load(os.path.join("imgs","bird2.png"))), pygame.transform.scale2x(pygame.image.load(os.path.join("imgs","bird3.png")))]
PIPE_IMG = pygame.transform.scale2x(pygame.image.load(os.path.join("imgs","pipe.png")))
BASE_IMG = pygame.transform.scale2x(pygame.image.load(os.path.join("imgs","base.png")))
BG_IMG = pygame.transform.scale2x(pygame.image.load(os.path.join("imgs","bg.png")))


In [16]:
STAT_FONT = pygame.font.SysFont("comicsans", 50)

In [17]:
# Bird Class
# Constants that we will use later

class Bird:
    IMGS = BIRD_IMGS
    MAX_ROTATION = 25 # How much the bird is going to tilt
    ROT_VEL = 20 # How much we are going to rotate each time we move the bird
    ANIMATION_TIME = 5 # How long will we show each bird animation

    def __init__(self,x,y):
        self.x = x
        self.y = y
        self.tilt = 0 # How much image is tilted
        self.tick_count = 0 # Physics of our bird
        self.vel = 0 # velocity because it isnt moving
        self.height = self.y #shouldnt reach max height
        self.img_count = 0  
        self.img = self.IMGS[0] # bird1.png
    
    def jump(self):
        self.vel = -10.5
        self.tick_count = 0 # When we last jumped
        self.height = self.y
        
    def move(self):
        self.tick_count += 1
        
        #for arc formation
        d = self.vel * self.tick_count + 1.5 * self.tick_count**2 
        
        # limit for d 
        if d >= 16:
            d = 16
            
        if d < 0:
            d -= 2
            
        self.y = self.y + d
        
        if d < 0 or self.y < self.height + 50:
            if self.tilt < self.MAX_ROTATION:
                self.tilt = self.MAX_ROTATION
        else: 
            if self.tilt > -90:
                self.tilt -= self.ROT_VEL
        
    def draw(self, win):
        self.img_count += 1
        
        if self.img_count < self.ANIMATION_TIME:
            self.img = self.IMGS[0]
        elif self.img_count <self.ANIMATION_TIME*2:
            self.img = self.IMGS[1]
        elif self.img_count <self.ANIMATION_TIME*3:
            self.img = self.IMGS[2]
        elif self.img_count <self.ANIMATION_TIME*4:
            self.img = self.IMGS[1]
        elif self.img_count == self.ANIMATION_TIME*4 + 1:
            self.img = self.IMGS[0]
            self.img_count = 0 
            
        if self.tilt <= -80:
            self.img = self.IMGS[1]
            self.img_count = self.ANIMATION_TIME*2
        
        rotated_image = pygame.transform.rotate(self.img, self.tilt)
        new_rect = rotated_image.get_rect(center = self.img.get_rect(topleft = (self.x, self.y)).center)
        win.blit(rotated_image, new_rect.topleft) # blit means draw
        
    def get_mask(self):
        return pygame.mask.from_surface(self.img)
        

In [18]:
class Pipe():
    GAP = 200
    VEL = 5
    
    def __init__(self,x):
        self.x = x
        self.height = 0
        self.gap = 100
        
        self.top = 0
        self.bottom = 0
        self.PIPE_TOP = pygame.transform.flip(PIPE_IMG, False, True)
        self.PIPE_BOTTOM = PIPE_IMG
        
        self.passed = False
        self.set_height()
        
    def set_height(self):
        self.height = random.randrange(50,450)
        self.top = self.height - self.PIPE_TOP.get_height()
        self.bottom = self.height + self.GAP
        
    def move(self):
        self.x -= self.VEL
    
    def draw(self, win):
        win.blit(self.PIPE_TOP, (self.x, self.top))
        win.blit(self.PIPE_BOTTOM, (self.x, self.bottom))
    
    def collide(self, bird):
        bird_mask = bird.get_mask()
        top_mask = pygame.mask.from_surface(self.PIPE_TOP)
        bottom_mask = pygame.mask.from_surface(self.PIPE_BOTTOM)
        
        top_offset = (self.x - bird.x, self.top - round(bird.y))
        bottom_offset = (self.x - bird.x, self.bottom - round(bird.y))
        
        b_point = bird_mask.overlap(bottom_mask, bottom_offset)
        t_point = bird_mask.overlap(bottom_mask, top_offset)
        
        if t_point or b_point:
            return True
        
        return False

In [19]:
class Base():
    VEL = 5 
    WIDTH = BASE_IMG.get_width()
    IMG = BASE_IMG
    
    def __init__(self,y):
        self.y = y
        self.x1 = 0
        self.x2 = self.WIDTH
    
    def move(self):
        self.x1 -= self.VEL
        self.x2 -= self.VEL
        
        if self.x1 + self.WIDTH < 0:
            self.x1 = self.x2 + self.WIDTH
        
        if self.x2 + self.WIDTH < 0:
            self.x2 = self.x1 + self.WIDTH
    
    def draw(self, win):
        win.blit(self.IMG, (self.x1, self.y))
        win.blit(self.IMG, (self.x2, self.y))

In [20]:
def draw_window(win, birds, pipes, base, score, gen):
    win.blit(BG_IMG, (0,0))
    
    for pipe in pipes:
        pipe.draw(win)
    
    text = STAT_FONT.render("Score: " + str(score), 1,(255,255,255))
    win.blit(text, (WIN_WIDTH - 10 - text.get_width(), 10))
    
    text = STAT_FONT.render("Gen: " + str(gen), 1,(255,255,255))
    win.blit(text, (10, 10))
    
    base.draw(win)
    for bird in birds:
        bird.draw(win)
        
    pygame.display.update()

In [21]:
def main(genomes, config):
    global GEN
    GEN += 1
    nets = []
    ge = []
    birds = []
    
    for _, g in genomes:
        net = neat.nn.FeedForwardNetwork.create(g, config)
        nets.append(net)
        birds.append(Bird(230, 350))
        g.fitness = 0
        ge.append(g)
    
    base = Base(730)
    pipes = [Pipe(600)]
    win = pygame.display.set_mode((WIN_WIDTH,WIN_HEIGHT))
    clock = pygame.time.Clock()
    
    score = 0
    
    run = True
    while run:
        clock.tick(30)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                pygame.quit()
                quit()
        
        pipe_ind = 0
        if len(birds) > 0:
            if len(pipes) > 1 and birds[0].x > pipes[0].x + pipes[0].PIPE_TOP.get_width():
                pipe_ind = 1
        else:
            run = False
            break
        
        for x,bird in enumerate(birds):
            bird.move()
            ge[x].fitness += 0.1
            
            output = nets[x].activate((bird.y, abs(bird.y - pipes[pipe_ind].height), abs(bird.y - pipes[pipe_ind].bottom)))
            
            #if you have more output neurons then you may have to check and run through  all of them 
            if output[0] > 0.5:
                bird.jump()
        
        #bird.move()
        add_pipe = False
        rem = []
        for pipe in pipes:
            for x, bird in enumerate(birds):
                if pipe.collide(bird):
                    ge[x].fitness -= 1
                    birds.pop(x)
                    nets.pop(x)
                    ge.pop(x)
             
                if not pipe.passed and pipe.x < bird.x:
                    pipe.passed = True
                    add_pipe = True
                
            if pipe.x + pipe.PIPE_TOP.get_width() < 0:
                rem.append(pipe)
                
            pipe.move()
           
        if add_pipe:
            score += 1
            for g in ge:
                g.fitness += 5
            pipes.append(Pipe(700))
            
        for r in rem:
            pipes.remove(r)
        
        for x, bird in enumerate(birds):
            if bird.y + bird.img.get_height() >= 730 or bird.y < 0:
                birds.pop(x)
                nets.pop(x)
                ge.pop(x)
                
        #if score > 50:
        #    break
        
        base.move()
        draw_window(win, birds, pipes, base, score, GEN)        

## AI -> Using NEAT algorithm

In [22]:
# Very important to think about what the input and out put is going to be for the algorithm

# Inputs -> Bird Y, Top Pipe, Bottom Pipe
# Outputs -> Jump / Dont Jump

In [23]:
# Activation Function -> TanH
# Population size -> 100 birds per generation #used 20
    #Gen 0 -> 100 birds 
    # (take best out of them) \|/
    #Gen 1 -> mutate best from Gen 0 to make 100 more birds
    # (take best out of them) \|/
    #.
    #.
    #.
# Fitness Function -> distance in x axis
# Max Generation -> 30

In [24]:
def run(config_path):
    
    config = neat.config.Config(neat.DefaultGenome,neat.DefaultReproduction, 
                                neat.DefaultSpeciesSet, neat.DefaultStagnation, 
                                config_path)

    p = neat.Population(config)
    
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    
    winner = p.run(main,50)
    

if __name__ == "__main__":
    local_dir = os.path.dirname("__file__")
    config_path = os.path.join(local_dir, "ConfiFile.txt")
    run(config_path)


 ****** Running generation 0 ****** 

Population's average fitness: 3.79500 stdev: 1.41791
Best fitness: 6.30000 - size: (1, 3) - species 1 - id 11
Average adjusted fitness: 0.374
Mean genetic distance 1.308, standard deviation 0.474
Population of 20 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0    20      6.3    0.374     0
Total extinctions: 0
Generation time: 2.752 sec

 ****** Running generation 1 ****** 

Population's average fitness: 4.93500 stdev: 3.63363
Best fitness: 17.20000 - size: (1, 3) - species 1 - id 29
Average adjusted fitness: 0.177
Mean genetic distance 1.107, standard deviation 0.435
Population of 20 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1    20     17.2    0.177     0
Total extinctions: 0
Generation time: 4.374 sec (3.563 average)

 ****** Running generation 2 ****** 



error: display Surface quit